Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Working With Antibodies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/12.00-Working-With-Antibodies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [RosettaAntibodyDesign](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/12.02-RosettaAntibodyDesign-RAbD.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/12.01-RosettaAntibody-Framework-and-SimpleMetrics.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# RosettaAntibody Framework
Keywords: CDRResidueSelector

## Overview
In this workshop we will learn how to use the RosettaAntibody framework.  The full RosettaAntibody (modeling) code is not available in PyRosetta, unfortunately - as it is based around an application. To use that, you will have to use either the ROSIE server, or the Rosetta application. 

For a full overview of the RosettaAntibody modeling application, see this paper: 
https://www.ncbi.nlm.nih.gov/pubmed/28125104

Snugdock, and H3 modeling component of RosettaAntibody are available here as movers. 

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

## Imports

Lets import the antibody namespace so we can start using it.  Take a look at the different modules that are a part of the antibody module.

Note that we can also do `from rosetta.protocols.antibody import *` in order to make accessing the enums much easier.  For the purpose of this workshop, we will use `antibody` to traverse the contents.  This makes it easier for you to use tab completion for exploration.

In [ ]:
#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

#Core Includes
from rosetta.core.select import residue_selector as selections

from rosetta.protocols import antibody


## Intitlialization 

Here, we will initialize a typical run of Rosetta. We could use the `-input_ab_scheme` option with `AHo_Scheme`, but we will learn to instead pass this to our main antibody framework code. 

In [ ]:
init('-use_input_sc -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -no_fconfig')

## Import and copy pose

Let's load an antibody - this this the same antibody we used to learn packing and design. :)

In [ ]:
#Import a pose
pose = pose_from_pdb("inputs/2r0l_1_1.pdb")
original_pose = pose.clone()

## AntibodyInfo

The main tool that we will use is the `AntibodyInfo` object.  This allows you to get a TON of information about the antibody to use in various custom protocols.  

Note that this antibody has already been renumbered using the PyIgClassify server.

Since we are not defining the numbering scheme and cdr definition during init, we will need to pass an Enum to the AntibodyInfo object.

In [ ]:
ab_info = antibody.AntibodyInfo(pose, antibody.AHO_Scheme, antibody.North)

Lets take a look at what AntibodyInfo prints

In [ ]:
print(ab_info)

**Isn't that AWESOME!!**  I think so.  But I wrote a lot of that code!  

Anyway, as you can see you can get a pretty fair bit of information out of the AntibodyInfo object.  In fact, most antibody-related code actually takes an AntibodyInfo object or constructs one from set numbering scheme, cdr definitions, and pose passed to it.  You will see this as we go.  

Note the north_cluster here.  This is useful in some modeling tasks, but becomes much more relevant during antibody design.  More information on what we mean by north_cluster can be found in this paper, if you want to read ahead a bit. https://www.ncbi.nlm.nih.gov/pubmed/21035459

## Basic AntibodyInfo Access
Now, lets use the AntibodyInfo class to get a bit of useful information out of our antibody.

In [ ]:
print("h1", ab_info.get_CDR_start(antibody.h1, pose))
print("h2", ab_info.get_CDR_end(antibody.h2, pose))

Now lets use these enums a bit more.  They go in order from 1 to 8, with 7 and 8 being CDR4 loops - also known as H3 loops.  We won't worry about them just yet.  

In [ ]:
for i in range(1, 7):
    print(i, ab_info.get_CDR_name(antibody.CDRNameEnum(i)))
    
for cdr in ['L1', 'l1', 'L2', 'l2', 'L3', 'H1', 'H2', 'H3']:
    print(cdr, str(ab_info.get_CDR_name_enum(cdr)))
          
print(str(antibody.h3))
print(int(antibody.h3))

Does this make enums a bit less confusing?  These are named integers.  The last function allows us to print either the actual cdr name enum or the integer from it.  The cool thing here is that we can loop through all of the CDRs just by using a range 1-6 and rosetta will understand it.  

Note that we convert the integer into a `CDRNameEnum` in the function.  If we are storing the cdr name enums as indexes to a dictionary or list, we don't need this.  That is simply for the C++ code to work properly. 

### AntibodyEnumManager
So we have seen that some of this code we can do directly within AntibodyInfo itself.  Cool. But what if we need something more advanced?  Lets use the class that actually does all this conversion.


In [ ]:
enum_manager = antibody.AntibodyEnumManager()
print(enum_manager.numbering_scheme_enum_to_string(antibody.AHO_Scheme))
print(enum_manager.cdr_definition_enum_to_string(antibody.North))
print(enum_manager.cdr_name_string_to_enum("H1"))
print(enum_manager.antibody_region_enum_to_string(antibody.framework_region))

Use the function, `get_region_or_residue` and `get_CDRNameEnum_of_residue` and the manager to traverse the antibody and get relevant regions of all residues in the pose

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### CDR Clusters

Use either the PyRosetta docs on AntibodyInfo, or the interactive notebook to use AntibodyInfo to get the length and cluster of L1.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

The CDRCluster object has a lot of information about a particular cluster.  Lets use it to get the normalized distance in degrees of the L1 cluster. 

In [ ]:
L1_cluster = ab_info.get_CDR_cluster(antibody.l1)
print(L1_cluster.normalized_distance_in_degrees())

Anything below 35 or 40 degrees is very close to the cluster center.  This is a structure with a very well-defined L1-11-1 loop - one of the most common L1 lengths and clusters.

### Numbering Scheme Translation
It may not seem like much, but numbering scheme translation is a very difficult thing to do without mistakes.   Rosetta now has this ability to make it much easier to understand antibody structural or sequence papers in a highly tested and fairly easy-to-use implementation.  Lets take a look.  We'll use `AntibodyInfo` and the `get_landmark_resnum()` function for this, but you could also use function `get_antibody_numbering_info()` that will give you all the conversions - though it is certainly a bit more tricky to use. 

#### Conserved Inter-Domain Cysteine

The conserved cysteine residue forming the intradomain disulfide bridge always carries the label "23" as in the IMGT numbering scheme, while according to Kabat, it was labeled L23 in Vk and Vl, H22 in VH.  Let's find this residue in our antibody. https://www.bioc.uzh.ch/plueckthun/antibody/Numbering/FR1/index.html

In [ ]:
rosetta_num = ab_info.get_landmark_resnum(pose, antibody.Kabat_Scheme, 'H', 22)

What is the chain and resnum in OUR Aho numbering scheme?  Is this a cysteine?  How about a disulfide?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Ok.  Cool.  Lets do the same thing for the Cysteine that is connected to this residue. 
In IMGT this is residue 104 on the heavy chain.  Lets do the same thing here.  Use tab completion for  `antibody.IMGT_Scheme` for the enum.  https://www.bioc.uzh.ch/plueckthun/antibody/Numbering/FR3a/index.html

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Once again, what is the residue in our AHO-numbered antibody?  Is it a Cysteine?  Is it disulfide bonded?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Sequence
Lets expore the sequence of this antibody

In [ ]:
ab_seq = ab_info.get_antibody_sequence()
print(ab_seq)

L1_seq = ab_info.get_CDR_sequence_with_stem(antibody.l1, pose)
print("L1", L1_seq)

for i in range(1, 7):
    cdr = antibody.CDRNameEnum(i)
    print(cdr, ab_info.get_CDR_sequence_with_stem(cdr, pose))

### Other AntibodyInfo functions
Use tab completion to find other useful functions.  This includes movemap, loops, and fold-tree creation for specific tasks.  With ResidueSelectors, this functionality is not quite as useful, but you should know that it is here.

### AntibodyInfo Deprecated Functions
All functions are fair-game, except these: `get_TaskFactory_AllCDRs` and `get_TaskFactory_OneCDR`  - This will be removed from AntibodyInfo as it is extremely specific to a particular antibody modeling task.

## Antibody Util and SimpleMetrics
Util functions in Rosetta are stored in the `util.hh` file in each directory that has one.  Within PyRosetta, when you import the namespace, these come with.  There are many that you should be aware of to make modeling and design tasks easier for custom protocols.

We will go through some examples here.

### Function: get_cdr_loops()
The get_cdr_loops function takes a vector1 bool of CDRs.  Use the Enums to set H3 and L3 to true.
Vector1 bool starts as all negative.

In [ ]:
h3_l3 = rosetta.utility.vector1_bool(6)
print(h3_l3)

h3_l3[antibody.h3] = True
h3_l3[antibody.l3] = True

#Here, we get cdr loops, and set the stem size to 2, 
# so we include 2 residues on either side of the CDR loop (called the stem), to help us in modeling.
h3_l3_loops = antibody.get_cdr_loops(ab_info, pose, h3_l3, 2)
print(h3_l3_loops)

### Function: select_epitope_residues()
We could use the NeighborhoodResidueSelector as you have used in the passed to get neighbors.  Instead, lets use a general function to get all the epitope residues within an 8 Angstrum distance of the paratope.

In [ ]:
epi_residues = antibody.select_epitope_residues(ab_info, pose, 8)
total=0
for i in range(1, len(epi_residues)+1):
    if epi_residues[i]:
        print(i)
        total+=1
print("Total Epitope Residues:", total)

So that was cool.  But lets the wonderful `ReturnResidueSubsetSelector` to take this `ResidueSubset` of the epitope residues and store the data as a `ResidueSelector`!

In [ ]:
epi_res_selector = selections.ReturnResidueSubsetSelector(epi_residues)

Now what?  Lets use some SimpleMetrics using the selector to calculate something about these epitope residues.

### SasaMetric, TotalEnergyMetric, SelectedResiduesPyMOLMetric

In [ ]:
import rosetta.core.simple_metrics.metrics as sm
sasa_metric = sm.SasaMetric(epi_res_selector)
print("\nSASA", sasa_metric.calculate(pose))

total_metric = sm.TotalEnergyMetric(epi_res_selector)
print("\nTOTAL RESIDUE ENERGY", total_metric.calculate(pose))

#Lets use a useful metric to select these residues in pymol
pymol_metric = sm.SelectedResiduesPyMOLMetric(epi_res_selector)
print("\nSELECTION", pymol_metric.calculate(pose))

Now lets see which of these residues are most buried in the interface and the residues which have the lowest energy. Note that this is not ddG - we would need to separate the chains for this.  We can use the `protocols.toolbox.rigid_body.translate` function to do that. 

Use the pymol selection (copy from select...) and lets take a look at them in PyMol.  Then run the code below.

### PerResidueSasaMetric

In [ ]:
import rosetta.core.simple_metrics.per_residue_metrics as residue_sm
import operator

res_sasa_metric = residue_sm.PerResidueSasaMetric()
res_sasa_metric.set_residue_selector(epi_res_selector)
per_res_sasa = res_sasa_metric.calculate(pose)
#print(per_res_sasa)

#Convert the Map to a Dictionary, which are essentially the same thing. 
for ele in sorted(per_res_sasa.items(), key=operator.itemgetter(1), reverse=False):
    print(ele)

Cool. So the most buried residues at the interface are 300, 303, 305.  Convert those to the PDB chain/num using PDBInfo and take a look at them in PyMOL.

### PerResidueEnergyMetric

In [ ]:
res_energy_metric = residue_sm.PerResidueEnergyMetric()
res_energy_metric.set_residue_selector(epi_res_selector)

per_res_energy = res_sasa_metric.calculate(pose)
#print(per_res_sasa)

#Convert the Map to a Dictionary, which are essentially the same thing. 
for ele in sorted(per_res_energy.items(), key=operator.itemgetter(1), reverse=False):
    print(ele[0], pose.pdb_info().pose2pdb(ele[0]), ele[1])

Wow!  Why is 48A so high in energy!?  This may be due to the fact that we are working with a crystal structure that has not been pre-relaxed using the pareto-optimal protocol.  Be sure when using PDBs from the data bank for production runs to do this, outputting about 10 models and selecting the lowest energy residue.  Or, you could use density to relax within the crystal denstiy.  Either works well. 

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0059004

## Other Useful Antibody Tools

### CDRResidueSelector

In [ ]:
from rosetta.protocols.antibody.residue_selector import *

cdr_selector = CDRResidueSelector(ab_info)
cdr_selector.set_cdrs(h3_l3)
sele = cdr_selector.apply(pose)
for i in range(1, len(sele)):
    if sele[i]:
        print(i, pose.pdb_info().pose2pdb(i))

### AntibodyRegionSelector
We can use the AntibodyRegionSelector to select a specific region:
`antigen_region`, `framework_region`, and `cdr_region`

In [ ]:
region_selector = AntibodyRegionSelector(ab_info)
region_selector.set_region(antibody.antigen_region)
sele = region_selector.apply(pose)

for i in range(1, len(sele)):
    if sele[i]:
        print(i, pose.pdb_info().pose2pdb(i))


### Other
-  **TaskOperations** - Antibody specific Task Operations will be covered in the next workshop
-  **SnugDock** - Snugdock is available in the `rosetta.protocols.antibody.snugdock` namespace.  Both the full protocol, `SnugDockProtocol` and the mover, `Snugdock` are available and easy to setup through code - but their run time is extremely long.
-  **AntibodyModelerProtocol** - this is the `Antibody_H3` app. Personally, I would use the Rosetta C++ application for this with specific options specified in the docs, however you can call this in PyRosetta.
-  **AntibodyCDRGrafter** This is the main grafting class used for RosettaAntibody and RosettaAntibodyDesign. Is is in the `protocols.antibody` namespace. Documentation on this mover can be found here (XML or code-level interface is available): https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/antibodies/AntibodyCDRGrafter   

## References
Please site these papers when using any of RosettaAntibody.

- J. Adolf-Bryfogle, O Kalyuzhniy, M Kubitz, B. D. Weitzner, X Hu, Y Adachi, W R. Schief, R L. Dunbrack Jr., 
    - "Rosetta Antibody Design (RAbD): A General Framework for Computational Antibody Design", PLOS Computational Biology (2018)

- B. D. Weitzner*, J. R. Jeliazkov*, S. Lyskov*, N. M. Marze, D. Kuroda, R. Frick, J. Adolf-Bryfogle, N. Biswas, R. L. Dunbrack Jr., and J. J. Gray, 
    - "Modeling and docking of antibody structures with Rosetta." Nature Protocols 12, 401–416 (2017)

- B. D. Weitzner, D. Kuroda, N. M. Marze, J. Xu & J. J. Gray, 
    - "Blind prediction performance of RosettaAntibody 3.0: Grafting, relaxation, kinematic loop modeling, and full CDR optimization." Proteins 82(8), 1611–1623 (2014)

- A. Sivasubramanian,* A. Sircar,* S. Chaudhury & J. J. Gray, 
    - "Toward high-resolution homology modeling of antibody Fv regions and application to antibody-antigen docking," Proteins 74(2), 497–514 (2009)

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

<!--NAVIGATION-->
< [Working With Antibodies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/12.00-Working-With-Antibodies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [RosettaAntibodyDesign](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/12.02-RosettaAntibodyDesign-RAbD.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/12.01-RosettaAntibody-Framework-and-SimpleMetrics.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>